In [1]:
import os
from src import RASPRoutines
RASP = RASPRoutines.RASP_Routines()
from src import AnalysisFunctions
A_F = AnalysisFunctions.Analysis_Functions()
from src import IOFunctions
IO = IOFunctions.IO_Functions()
from src import PlottingFunctions
plots = PlottingFunctions.Plotter()
import numpy as np
import matplotlib.pyplot as plt

In [2]:
analysis_file = '/home/jbeckwith/Documents/Dropbox (Cambridge University)/Chemistry/Lee/Code/Python/pyRASP/example_images_analysis/spot_analysis.csv'

In [3]:
threshold = 4000
protein_string='C0'
cell_string='C1'

In [5]:
RASP.colocalise_with_threshold(analysis_file, threshold, cell_string, protein_string, calc_clr=True)

/home/jbeckwith/.virtualenvs/pyRASP/lib/python3.10/site-packages/numpy/core/numeric.py:407: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(res, fill_value, casting='unsafe')


5988
499
6288
524
6300
525
544
544
631
631
586
586
591
591
680
680
658
658
7632
636
7908
659
8844
737
8580
715
7860
655


ValueError: Length of values (63090) does not match length of index (8640)

In [ ]:
folder = r'/home/jbeckwith/Documents/Dropbox (Cambridge University)/Chemistry/Lee/Code/Python/pyRASP/negative_control'
files_list = os.listdir(folder)
imtype=".tif"; gsigma=1.4; rwave=2.0; accepted_ratio=1

files = np.sort([e for e in files_list if imtype in e])
k1, k2 = A_F.create_kernel(gsigma, rwave)  # create image processing kernels
rdl = [np.inf, 0.0, 0.0]
thres = 0.05
r1_neg_forplot = {}  # generate dictionaries for plotting
r2_neg_forplot = {}
intensity_forplot = {}


In [ ]:
        for i in np.arange(len(files)):
            file_path = os.path.join(folder, files[i])
            image = IO.read_tiff_tophotons(file_path)
            if len(image.shape) < 3:
                dl_mask, centroids, radiality, large_mask = A_F.compute_image_props(
                    image,
                    k1,
                    k2,
                    thres,
                    10000.0,
                    RASP.areathres,
                    rdl,
                    RASP.d,
                    calib=True,
                )
                
                r1_neg_forplot[i] = radiality[:, 0]
                r2_neg_forplot[i] = radiality[:, 1]
                estimated_intensity, estimated_background, estimated_background_perpixel = (
                    A_F.estimate_intensity(image, np.asarray(centroids, dtype=int))
                            )
                intensity_forplot[i] = estimated_intensity
            else:
                z_planes = RASP.get_infocus_planes(image, k1)
                z_planes = np.arange(z_planes[0], z_planes[-1])
                if len(z_planes) != 0:  # if there are images we want to analyse
                    dl_mask, centroids, radiality, large_mask = A_F.compute_image_props(
                        image,
                        k1,
                        k2,
                        thres,
                        10000.0,
                        RASP.areathres,
                        rdl,
                        RASP.d,
                        z_planes=z_planes,
                        calib=True,
                    )
                    for z in enumerate(z_planes):
                        r1_neg_forplot[i, z[0]] = radiality[z[1]][:, 0]
                        r2_neg_forplot[i, z[0]] = radiality[z[1]][:, 1]
                        estimated_intensity, estimated_background, estimated_background_perpixel = (
                        A_F.estimate_intensity(image[:, :, z[1]], np.asarray(centroids[z[1]], dtype=int))
                            )
                        intensity_forplot[i, z[0]] = estimated_intensity



In [ ]:
keys = list(intensity_forplot.keys())
for i, key in enumerate(keys):
    todel = np.isnan(intensity_forplot[key])
    intensity_forplot[key] = intensity_forplot[key][~todel]
    r1_neg_forplot[key] = r1_neg_forplot[key][~todel]
    r2_neg_forplot[key] = r2_neg_forplot[key][~todel]


In [ ]:
for i, key in enumerate(keys):
    if i == 0:
        intensity = np.array(intensity_forplot[key])
        r1_neg = np.array(r1_neg_forplot[key])
        r2_neg = np.array(r2_neg_forplot[key])
    else:
        intensity = np.hstack([intensity, np.array(intensity_forplot[key])])
        r1_neg = np.hstack([r1_neg, np.array(r1_neg_forplot[key])])
        r2_neg = np.hstack([r2_neg, np.array(r2_neg_forplot[key])])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(intensity, r1_neg, r2_neg, alpha=0.5)


ax.set_xlabel('intensity (photons)')
ax.set_ylabel('Flatness')
ax.set_zlabel('Integrated Gradient')

plt.tight_layout()
plt.savefig('Testing_intensity.svg', format='svg')
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8,8))

axs[0,1].hist2d(intensity, r1_neg, bins=(50,50))
axs[0, 1].set_title('intensity vs flatness')
axs[1,0].hist2d(intensity, r2_neg, bins=(50,50))
axs[1, 0].set_title('intensity vs integrated gradient')
axs[0,0].hist2d(r1_neg, r2_neg, bins=(50,50))
axs[0, 0].set_title('flatness vs integrated gradient')
axs[1,1].hist2d(intensity, intensity, bins=(50,50))
axs[1, 1].set_title('intensity vs intensity')

plt.savefig('Testing_intensity.svg', format='svg')

plt.show()